
JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,

    Find the 10 countries with most projects
    Find the top 10 major project themes (using column 'mjtheme_namecode')
    In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.



In [3]:
#import the relevant libraries and packages
import pandas as pd
import json
from pandas.io.json import json_normalize

#as cell is static, also define reference to local file
world_bank_json = r'''C:\\Users\\Garrick\\Documents\\Springboard\\Curriculum\\Section 5 - Data Wrangling\\Project Work on JSON Based Data Exercises and Submit on Your Github\\data_wrangling_json\\data_wrangling_json\\data\\world_bank_projects.json'''

Objective 1:  Find the 10 countries with most projects

In [4]:
#opening the JSON file in "read-only" mode using a context manager
with open(world_bank_json, 'r') as json_file:
    wb_json_data = json.load(json_file)
    
#checking to see the structure of the loaded World Bank data
print(type(wb_json_data))

#as the object appears to be a list, let's examine the number of items in the list
print(len(wb_json_data))

#diving further what each element contains
print(type(wb_json_data[0]))

#appears object is a list of dictionaries/key-value pairs. let's further explore the contents...
print(wb_json_data[0].keys())

#as a result, our object is a list of 500 projects with the below dictionary keys

<class 'list'>
500
<class 'dict'>
dict_keys(['sector', 'supplementprojectflg', 'projectfinancialtype', 'prodline', 'mjtheme', 'idacommamt', 'impagency', 'project_name', 'mjthemecode', 'closingdate', 'totalcommamt', 'id', 'mjsector_namecode', 'docty', 'sector1', 'lendinginstr', 'countrycode', 'sector2', 'totalamt', 'mjtheme_namecode', 'boardapprovaldate', 'countryshortname', 'sector4', 'prodlinetext', 'productlinetype', 'regionname', 'status', 'country_namecode', 'envassesmentcategorycode', 'project_abstract', 'approvalfy', 'projectdocs', 'lendprojectcost', 'lendinginstrtype', 'theme1', 'grantamt', 'themecode', 'borrower', 'sectorcode', 'sector3', 'majorsector_percent', 'board_approval_month', 'theme_namecode', 'countryname', 'url', 'source', 'projectstatusdisplay', 'ibrdcommamt', 'sector_namecode', '_id'])


In [5]:
# use normalization to create a table and examine the values
print(json_normalize(wb_json_data).head(10))

                   _id.$oid approvalfy board_approval_month  \
0  52b213b38594d8a2be17c780       1999             November   
1  52b213b38594d8a2be17c781       2015             November   
2  52b213b38594d8a2be17c782       2014             November   
3  52b213b38594d8a2be17c783       2014              October   
4  52b213b38594d8a2be17c784       2014              October   
5  52b213b38594d8a2be17c785       2014              October   
6  52b213b38594d8a2be17c786       2014              October   
7  52b213b38594d8a2be17c787       2014              October   
8  52b213b38594d8a2be17c788       2014              October   
9  52b213b38594d8a2be17c789       2014              October   

      boardapprovaldate                                 borrower  \
0  2013-11-12T00:00:00Z  FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA   
1  2013-11-04T00:00:00Z                    GOVERNMENT OF TUNISIA   
2  2013-11-01T00:00:00Z   MINISTRY OF FINANCE AND ECONOMIC DEVEL   
3  2013-10-31T00:00:00Z   MIN. OF 

In [6]:
#ensure there are no null values in the countryshortname column
wb_transform = json_normalize(wb_json_data)
print(wb_transform['countryshortname'])

#As there are 500 values for 'countryshortname' and 500 rows in the data, we can assert there are no null values in this column.
#This also looks to be the column and data to answer Question 1. Perfect! 

0                              Ethiopia
1                               Tunisia
2                                Tuvalu
3                    Yemen, Republic of
4                               Lesotho
5                                 Kenya
6                                 India
7                                 China
8                                 India
9                               Morocco
10                          South Sudan
11                                India
12                                Ghana
13                          Timor-Leste
14                               Jordan
15                                Samoa
16                                Samoa
17                           Madagascar
18                             Cambodia
19                              Morocco
20                      Kyrgyz Republic
21                                Nepal
22                               Jordan
23                           Tajikistan
24                           Azerbaijan


In [7]:
#convert JSON data into DataFrame
wb_json_df = pd.read_json(world_bank_json)

#count the instances of each 'countryshortname' value
wb_json_country_count = wb_json_df['countryshortname'].value_counts()
print('10 Countries with the most projects:')
print(wb_json_country_count.head(10))

10 Countries with the most projects:
China                 19
Indonesia             19
Vietnam               17
India                 16
Yemen, Republic of    13
Nepal                 12
Bangladesh            12
Morocco               12
Africa                11
Mozambique            11
Name: countryshortname, dtype: int64


Objective 2: Find the top 10 major project themes (using column 'mjtheme_namecode')

In [8]:
#create series on column 'mjtheme_namecode'
wb_json_theme_df = wb_json_df['mjtheme_namecode']
print(wb_json_theme_df.shape)

#checking amount of NaN values
print(wb_json_theme_df.dropna().shape)

#all rows appear to have data.  However, as per prompt, some values are missing

(500,)
(500,)


In [9]:
#getting a sense of the objects in each row
wb_json_theme_df[0]

[{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': ''}]

In [10]:
#finding main project themes without additional data cleaning

from itertools import chain
parsed = (json.loads((wb_json_df.mjtheme_namecode.to_json(orient='values'))))
new_parsed_df = pd.DataFrame(list(chain.from_iterable(parsed)))

#now applying the .value_counts method to count up instances
theme_count = new_parsed_df['name'].value_counts()
print('10 Major Project themes:')
print(theme_count.head(10))

10 Major Project themes:
Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64


In [11]:
#map out the theme names to codes
name_code_dict = {}
for row in new_parsed_df.itertuples():
    if row[2] != '':
        name_code_dict[row[1]] = row[2]
        
name_code_dict

{'1': 'Economic management',
 '10': 'Rural development',
 '11': 'Environment and natural resources management',
 '2': 'Public sector governance',
 '3': 'Rule of law',
 '4': 'Financial and private sector development',
 '5': 'Trade and integration',
 '6': 'Social protection and risk management',
 '7': 'Social dev/gender/inclusion',
 '8': 'Human development',
 '9': 'Urban development'}

In [12]:
# Fill in missing theme names using the name dictionary
for row in new_parsed_df.itertuples():
    if row[2] == '':
        new_parsed_df.set_value(row[0], 'name', name_code_dict[row[1]])

In [13]:
print('Top 10 major project themes with the missing names filled in:')
print(new_parsed_df.name.value_counts().head(10))

Top 10 major project themes with the missing names filled in:
Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Name: name, dtype: int64


Compared to the data without the missing "name" values filled in given their respective "code", we see that the top 10 themes are largely unchanged.  However, we do see the number of instances is considerably different.  For example, the top the theme "Environment and natural resources management" went from 223 times to 250.  

This finding further indicates that the 122 instances where no theme name existed, was actually an amalgamation of projects where the data had a code but the 'name' field missing.  As a result, we expect the 122 projects are actually dispersed among the 11 themes.  

Furthermore, the new theme among the top 10 is "Economic Development" which was not previously uncovered. While the 'Rule of Law' theme was absent from both Top 10 findings.